In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
XML_input = input() 

In [ ]:
DF_output = input()

In [ ]:
tree = ET.parse(XML_input)
root = tree.getroot()

In [ ]:
data = []
for node in root:
    article_data = {}
    if node.tag == 'PubmedArticle':
        article_data['Type'] = 'Article'
    elif node.tag == 'PubmedBookArticle':
        article_data['Type'] = 'Book Article'
    else:
        continue

    pmid_node = node.find('./MedlineCitation/PMID')
    article_data['PMID'] = pmid_node.text.strip() if pmid_node is not None else None

    journal_node = node.find('./MedlineCitation/Article/Journal/Title')
    article_data['Journal'] = journal_node.text.strip() if journal_node is not None else None

    article_title_node = node.find('./MedlineCitation/Article/ArticleTitle')
    article_data['ArticleTitle'] = article_title_node.text.strip() if article_title_node is not None else None

    abstract_node = node.find('./MedlineCitation/Article/Abstract')
    if abstract_node is not None:
        abstract_text = ''
        for elem in abstract_node.iter():
            if elem.text:
                abstract_text += elem.text.strip() + ' '
        article_data['Abstract'] = abstract_text.strip()
    else:
        article_data['Abstract'] = None

    authors = []
    for author in node.findall('./MedlineCitation/Article/AuthorList/Author'):
        author_info = {}
        forename_node = author.find('./ForeName')
        lastname_node = author.find('./LastName')
        if forename_node is not None and lastname_node is not None:
            author_info['Name'] = forename_node.text.strip() + ' ' + lastname_node.text.strip()
        else:
            continue

        affiliations = []
        for affiliation in author.findall('./AffiliationInfo/Affiliation'):
            affiliations.append(affiliation.text.strip())

        author_info['Affiliation'] = affiliations
        authors.append(author_info)

    article_data['Authors'] = authors

    mesh_headings = []
    for mesh_heading in node.findall('./MedlineCitation/MeshHeadingList/MeshHeading'):
        descriptor_name = mesh_heading.find('./DescriptorName').text.strip()
        mesh_headings.append(descriptor_name)
    article_data['MeshHeadings'] = ', '.join(mesh_headings) if mesh_headings else None

    chemicals = []
    for chemical in node.findall('./MedlineCitation/ChemicalList/Chemical'):
        chemical_name = chemical.find('./NameOfSubstance').text.strip()
        chemicals.append(chemical_name)
    article_data['Chemicals'] = ', '.join(chemicals) if chemicals else None

    publication_types = []
    for publication_type in node.findall('./MedlineCitation/Article/PublicationTypeList/PublicationType'):
        publication_types.append(publication_type.text.strip())
    article_data['PublicationTypes'] = ', '.join(publication_types) if publication_types else None

    pub_date_node = node.find('./MedlineCitation/Article/Journal/JournalIssue/PubDate/Year')
    if pub_date_node is not None:
        article_data['PublicationDate'] = pub_date_node.text.strip()
    else:
        article_data['PublicationDate'] = None

    language_node = node.find('./MedlineCitation/Article/Language')
    article_data['Language'] = language_node.text.strip() if language_node is not None else None

    keywords = []
    for keyword in node.findall('./MedlineCitation/KeywordList/Keyword'):
        keywords.append(keyword.text.strip())
    article_data['Keywords'] = ', '.join(keywords) if keywords else None

    data.append(article_data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("second extraction, full affiliation.csv")